In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9

In [2]:
%pip install -r requirements.txt -q


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
%cd ..

In [13]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

In [ ]:
import albumentations as A
import cv2
import os
import torch
import numpy as np
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.7),
    A.GaussianBlur(blur_limit=(3, 7), p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    A.RandomGamma(gamma_limit=(80, 120), p=0.7),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.7),
    A.ToGray(p=0.7),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.7),
    ToTensorV2()
])

def read_yolo_labels(label_path):
    with open(label_path, 'r') as f:
        labels = [line.strip() for line in f.readlines()]
    return labels

def write_yolo_labels(label_path, labels):
    with open(label_path, 'w') as f:
        for label in labels:
            f.write(f"{label}\n")

def augment_image_on_cuda(image):
    image_gpu = torch.from_numpy(image).cuda().float().permute(2, 0, 1) / 255.0
    image_cpu = image_gpu.permute(1, 2, 0).cpu().numpy() * 255.0
    image_cpu = image_cpu.astype(np.uint8)
    
    return image_cpu

def main(image_path, label_path, output_image_dir, output_label_dir, augmented_num):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    labels = read_yolo_labels(label_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    for i in range(augmented_num):
        transformed_image = augment_image_on_cuda(image)
        augmented = transform(image=transformed_image)
        transformed_image = augmented['image'].permute(1, 2, 0).cpu().numpy() * 255.0

        transformed_image = transformed_image.astype(np.uint8)
        transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR)

        output_image_path = os.path.join(output_image_dir, f"{base_name}_{i}.jpg")
        output_label_path = os.path.join(output_label_dir, f"{base_name}_{i}.txt")

        cv2.imwrite(output_image_path, transformed_image)
        write_yolo_labels(output_label_path, labels)

if __name__ == "__main__":
    # Define directories
    input_image_dir = 'velding_defects'
    input_label_dir = 'velding_defects'
    output_image_dir = 'augmented_velding_defects'
    output_label_dir = 'augmented_velding_defects'
    
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)

    for image_name in os.listdir(input_image_dir):
        if image_name.endswith('.jpg'):
            base_name = os.path.splitext(image_name)[0]
            image_path = os.path.join(input_image_dir, image_name)
            label_path = os.path.join(input_label_dir, base_name + '.txt')

            if os.path.exists(label_path):
                main(image_path, label_path, output_image_dir, output_label_dir, augmented_num=2)


In [8]:
import os
import shutil
import random

base_dir = 'velding_defects'  # Путь к датасету
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# Необходимые папки
image_train_dir = os.path.join(images_dir, 'train')
image_val_dir = os.path.join(images_dir, 'val')
label_train_dir = os.path.join(labels_dir, 'train')
label_val_dir = os.path.join(labels_dir, 'val')

# Создание папок
os.makedirs(image_train_dir, exist_ok=True)
os.makedirs(image_val_dir, exist_ok=True)
os.makedirs(label_train_dir, exist_ok=True)
os.makedirs(label_val_dir, exist_ok=True)

# Получение списка всех файлов изображений
image_files = [f for f in os.listdir(base_dir) if f.endswith('.jpg')]

# Перемешивание и разделение файлов
random.shuffle(image_files)
split_index = int(0.7 * len(image_files))

train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Функция перемещения файлов, если существует соответствующая метка
def move_files(file_list, target_image_dir, target_label_dir):
    for file in file_list:
        image_path = os.path.join(base_dir, file)
        label_name = os.path.splitext(file)[0] + '.txt'
        label_path = os.path.join(base_dir, label_name)
        
        if os.path.exists(label_path):
            shutil.move(image_path, target_image_dir)
            shutil.move(label_path, target_label_dir)
        else:
            print(f"Label for {file} not found, skipping.")

# Перемещение файлов в соответствующие каталоги
move_files(train_files, image_train_dir, label_train_dir)
move_files(val_files, image_val_dir, label_val_dir)

print("Files have been moved successfully.")

Files have been moved successfully.


In [9]:
import os
from pathlib import Path

# Укажите базовую директорию
base_dir = Path('velding_defects')
train_images_dir = base_dir / 'images' / 'train'
val_images_dir = base_dir / 'images' / 'val'

# Пути к файлам train.txt и val.txt
train_txt_path = base_dir / 'train.txt'
val_txt_path = base_dir / 'val.txt'

# Функция для записи списка файлов с полными путями
def write_file_list(file_list, file_path):
    with open(file_path, 'w') as f:
        for file in file_list:
            f.write(str(file) + '\n')

# Функция для получения списка всех .jpg изображений в директории
def get_jpg_files(directory):
    return list(directory.rglob('*.jpg'))

# Получение всех .jpg изображений в тренировочных и валидационных директориях
train_images = get_jpg_files(train_images_dir)
val_images = get_jpg_files(val_images_dir)

# Запись списков файлов в train.txt и val.txt с полными путями
write_file_list(train_images, train_txt_path)
write_file_list(val_images, val_txt_path)

print(f"Файлы {train_txt_path} и {val_txt_path} успешно созданы.")

Файлы velding_defects/train.txt и velding_defects/val.txt успешно созданы.


In [ ]:
%cd yolov9

In [14]:
!python3 -m torch.distributed.run --nproc_per_node 2 train_dual.py \
--batch 32 --epochs 30 --img 640 --device 0,1 --min-items 0 --close-mosaic 5 \
--data velding_defects/data.yaml \
--weights /home/jupyter/work/resources/weights/yolov9-e.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
2024-06-15 22:04:48.432122: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 22:04:48.432122: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 22:04:50.152300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [4]:
!python3 -m torch.distributed.run --nproc_per_node 2 train_dual.py \
--batch 32 --epochs 22 --img 640 --device 0,1 --min-items 0 --close-mosaic 5 \
--data velding_defects/data.yaml \
--weights /home/jupyter/work/resources/yolov9/runs/train/exp12/weights/best.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
2024-06-16 00:20:21.217490: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 00:20:21.217493: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 00:20:22.926929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [9]:
!python3 -m torch.distributed.run --nproc_per_node 2 train_dual.py \
--batch 32 --epochs 25 --img 640 --device 0,1 --min-items 0 --close-mosaic 0 \
--data velding_defects/data.yaml \
--weights /home/jupyter/work/resources/yolov9/runs/train/exp13/weights/best.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml

2024-06-16 01:28:20.936423: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 01:28:20.936423: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 01:28:21.003268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 01:28:21.003292: I tensorflow/core/platform/cpu_feature_guard.cc:182]

In [ ]:
!python3 -m torch.distributed.run --nproc_per_node 2 train_dual.py \
--batch 32 --epochs 20 --img 640 --device 0,1 --min-items 0 --close-mosaic 0 \
--data velding_defects/data.yaml \
--weights /home/jupyter/work/resources/yolov9/runs/train/exp14/weights/best.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
2024-06-16 03:11:41.474366: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 03:11:41.474366: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 03:11:41.535008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [6]:
!python3 -m torch.distributed.run --nproc_per_node 2 train_dual.py \
--batch 32 --epochs 20 --img 640 --device 0,1 --min-items 0 --close-mosaic 0 \
--data velding_defects/data.yaml \
--weights /home/jupyter/work/resources/yolov9/runs/train/exp16/weights/best.pt \
--cfg models/detect/yolov9-e.yaml \
--hyp hyp.scratch-high.yaml

2024-06-16 04:16:25.597498: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 04:16:25.597498: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-16 04:16:25.664225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 04:16:25.664225: I tensorflow/core/platform/cpu_feature_guard.cc:182]

In [12]:
%cd ..

/home/jupyter/work/resources


In [13]:
from yolov9.detect_dual import run

run(
    weights='/home/jupyter/work/resources/yolov9/runs/train/exp4/weights/best.pt',
    conf_thres=0.1,
    imgsz=(640,640),
    source='/home/jupyter/work/resources/test_dataset',
    save_txt=True,
    nosave=True,
    device='0'
)

YOLO 🚀 2024-6-15 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50707518 parameters, 0 gradients, 236.7 GFLOPs
image 1/363 /home/jupyter/work/resources/test_dataset/0.jpg: 384x640 1 geo, 40.9ms
image 2/363 /home/jupyter/work/resources/test_dataset/1 (1).jpg: 384x640 2 ints, 3 geos, 21.4ms
image 3/363 /home/jupyter/work/resources/test_dataset/1 (101).jpg: 384x640 2 geos, 21.7ms
image 4/363 /home/jupyter/work/resources/test_dataset/1 (102).jpg: 384x640 4 geos, 21.6ms
image 5/363 /home/jupyter/work/resources/test_dataset/1 (103).jpg: 384x640 1 int, 5 geos, 21.3ms
image 6/363 /home/jupyter/work/resources/test_dataset/1 (104).jpg: 384x640 1 int, 4 geos, 21.2ms
image 7/363 /home/jupyter/work/resources/test_dataset/1 (107).jpg: 384x640 1 int, 1 geo, 20.8ms
image 8/363 /home/jupyter/work/resources/test_dataset/1 (108).jpg: 384x640 1 int, 1 geo, 20.6ms
image 9/363 /home/jupyter/work/resources/test_dataset/1 (109).jpg: 38

In [14]:
%cd yolov9

/home/jupyter/work/resources/yolov9


In [15]:
import os

data = []
folder_path = 'runs/detect/exp38/labels'

for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        image_name = file_name.replace(".txt", ".jpg")
        file_path = os.path.join(folder_path, file_name)
            
        with open(file_path, "r") as file:
            lines = file.readlines()
                
            for line in lines:
                parts = line.strip().split()
                class_id, rel_x, rel_y, width, height = parts
                data.append((image_name, int(class_id), float(rel_x), float(rel_y), float(width), float(height)))

In [16]:
import pandas as pd

df = pd.DataFrame(data, columns=['filename','class_id','rel_x','rel_y','width','height'])

df.to_csv("sumbission20.csv", index=False, sep=";")

In [10]:
!python3 val_dual.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data velding_defects/data.yaml \
--weights runs/train/exp14/weights/best.pt\

val_dual: data=velding_defects/data.yaml, weights=['runs/train/exp14/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 2024-6-15 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)

Fusing layers... 
yolov9-e summary: 839 layers, 68553982 parameters, 0 gradients, 240.7 GFLOPs
val: Scanning /home/jupyter/work/resources/yolov9/velding_defects/val.cache... 1046 images, 137 backgrounds, 0 corrupt: 100%|██████████| 1046/1046 00:00
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 33/33 00:28
                   all       1046       3051      0.947       0.93      0.964      0.609
                   adj       1046       1338      0.921      0.925